In [1]:
using CSV, DataFrames, JuMP, Gurobi, Plots, HDF5, JLD

In [2]:
# Some constants for the model
alpha = 0.1 # Hospitalization rate for non-vaccinated
beta = 0.02 ;# Hospitalization rate for vaccinated 
gamma = 0.1 # State can only vaccinate 10% of population per week
file_name = "solution_real" ; # JLD file to save the solution variables in for later analysis 

In [34]:
data_folder = 'real_world_data' 
# Load csv files
case_file = "case_data_real.csv"
population_file = "population_data_real.csv"
vaccine_file = "vaccine_data_real.csv"

case_data = CSV.read(case_file, DataFrame)
population_data = CSV.read(population_file, DataFrame)
vaccine_data = CSV.read(vaccine_file, DataFrame)

#extract 'population' column from population_data
population_data = population_data[:,2] 

P = Vector(population_data)
C = Matrix(case_data)
V = Matrix(vaccine_data)
# convert population data to matrix

C =C'
V = V[:,1]

println("C: ", size(C))
println("P: ", size(P))
println("V: ", size(V))

C: (50, 52)
P: (50,)
V: (52,)


In [160]:
n_states = 1:size(case_data, 2)
n_weeks = 1:size(case_data, 1) 
n_weeks_2 = 1:size(case_data, 1)-1 

total_states = length(n_states) 
total_weeks = length(n_weeks) 
println("Number of states: ", n_states)
println("Number of weeks: ", n_weeks)

Number of states: 1:50
Number of weeks: 1:20


In [161]:
model = Model(Gurobi.Optimizer)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [162]:
@variable(model, X[n_states, n_weeks] >= 0);
X

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 1:20
And data, a 50×20 Matrix{VariableRef}:
 X[1,1]   X[1,2]   X[1,3]   X[1,4]   X[1,5]   …  X[1,18]   X[1,19]   X[1,20]
 X[2,1]   X[2,2]   X[2,3]   X[2,4]   X[2,5]      X[2,18]   X[2,19]   X[2,20]
 X[3,1]   X[3,2]   X[3,3]   X[3,4]   X[3,5]      X[3,18]   X[3,19]   X[3,20]
 X[4,1]   X[4,2]   X[4,3]   X[4,4]   X[4,5]      X[4,18]   X[4,19]   X[4,20]
 X[5,1]   X[5,2]   X[5,3]   X[5,4]   X[5,5]      X[5,18]   X[5,19]   X[5,20]
 X[6,1]   X[6,2]   X[6,3]   X[6,4]   X[6,5]   …  X[6,18]   X[6,19]   X[6,20]
 X[7,1]   X[7,2]   X[7,3]   X[7,4]   X[7,5]      X[7,18]   X[7,19]   X[7,20]
 X[8,1]   X[8,2]   X[8,3]   X[8,4]   X[8,5]      X[8,18]   X[8,19]   X[8,20]
 X[9,1]   X[9,2]   X[9,3]   X[9,4]   X[9,5]      X[9,18]   X[9,19]   X[9,20]
 X[10,1]  X[10,2]  X[10,3]  X[10,4]  X[10,5]     X[10,18]  X[10,19]  X[10,20]
 X[11,1]  X[11,2]  X[11,3]  X[11,4]  X[11,5]  …  X[11,18]  X[11,19]  X[11,20]
 X[

In [163]:
@variable(model, Y[n_states, n_weeks] >= 0);
Y

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 1:20
And data, a 50×20 Matrix{VariableRef}:
 Y[1,1]   Y[1,2]   Y[1,3]   Y[1,4]   Y[1,5]   …  Y[1,18]   Y[1,19]   Y[1,20]
 Y[2,1]   Y[2,2]   Y[2,3]   Y[2,4]   Y[2,5]      Y[2,18]   Y[2,19]   Y[2,20]
 Y[3,1]   Y[3,2]   Y[3,3]   Y[3,4]   Y[3,5]      Y[3,18]   Y[3,19]   Y[3,20]
 Y[4,1]   Y[4,2]   Y[4,3]   Y[4,4]   Y[4,5]      Y[4,18]   Y[4,19]   Y[4,20]
 Y[5,1]   Y[5,2]   Y[5,3]   Y[5,4]   Y[5,5]      Y[5,18]   Y[5,19]   Y[5,20]
 Y[6,1]   Y[6,2]   Y[6,3]   Y[6,4]   Y[6,5]   …  Y[6,18]   Y[6,19]   Y[6,20]
 Y[7,1]   Y[7,2]   Y[7,3]   Y[7,4]   Y[7,5]      Y[7,18]   Y[7,19]   Y[7,20]
 Y[8,1]   Y[8,2]   Y[8,3]   Y[8,4]   Y[8,5]      Y[8,18]   Y[8,19]   Y[8,20]
 Y[9,1]   Y[9,2]   Y[9,3]   Y[9,4]   Y[9,5]      Y[9,18]   Y[9,19]   Y[9,20]
 Y[10,1]  Y[10,2]  Y[10,3]  Y[10,4]  Y[10,5]     Y[10,18]  Y[10,19]  Y[10,20]
 Y[11,1]  Y[11,2]  Y[11,3]  Y[11,4]  Y[11,5]  …  Y[11,18]  Y[11,19]  Y[11,20]
 Y[

In [164]:
@variable(model, W[n_states, n_weeks] >= 0);
W

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 1:20
And data, a 50×20 Matrix{VariableRef}:
 W[1,1]   W[1,2]   W[1,3]   W[1,4]   W[1,5]   …  W[1,18]   W[1,19]   W[1,20]
 W[2,1]   W[2,2]   W[2,3]   W[2,4]   W[2,5]      W[2,18]   W[2,19]   W[2,20]
 W[3,1]   W[3,2]   W[3,3]   W[3,4]   W[3,5]      W[3,18]   W[3,19]   W[3,20]
 W[4,1]   W[4,2]   W[4,3]   W[4,4]   W[4,5]      W[4,18]   W[4,19]   W[4,20]
 W[5,1]   W[5,2]   W[5,3]   W[5,4]   W[5,5]      W[5,18]   W[5,19]   W[5,20]
 W[6,1]   W[6,2]   W[6,3]   W[6,4]   W[6,5]   …  W[6,18]   W[6,19]   W[6,20]
 W[7,1]   W[7,2]   W[7,3]   W[7,4]   W[7,5]      W[7,18]   W[7,19]   W[7,20]
 W[8,1]   W[8,2]   W[8,3]   W[8,4]   W[8,5]      W[8,18]   W[8,19]   W[8,20]
 W[9,1]   W[9,2]   W[9,3]   W[9,4]   W[9,5]      W[9,18]   W[9,19]   W[9,20]
 W[10,1]  W[10,2]  W[10,3]  W[10,4]  W[10,5]     W[10,18]  W[10,19]  W[10,20]
 W[11,1]  W[11,2]  W[11,3]  W[11,4]  W[11,5]  …  W[11,18]  W[11,19]  W[11,20]
 W[

In [165]:
@objective(model, Min, sum(sum(Y[i,j] for i in n_states) for j in n_weeks)) ; 

In [166]:
@constraint(model, hospitalizations[i in n_states,j in n_weeks], Y[i,j] == alpha * C[i,j] * (1 - W[i,j]) + beta * C[i,j] * W[i,j]) 

2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 1:20
And data, a 50×20 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 hospitalizations[1,1] : Y[1,1] + 41.84 W[1,1] = 52.300000000000004                  …  hospitalizations[1,20] : Y[1,20] + 1767.7600000000002 W[1,20] = 2209.7000000000003
 hospitalizations[2,1] : Y[2,1] + 29.68 W[2,1] = 37.1                                   hospitalizations[2,20] : Y[2,20] + 369.28000000000003 W[2,20] = 461.6
 hospitalizations[3,1] : Y[3,1] = 0                                                     hospitalizations[3,20] : Y[3,20] + 1520 W[3,20] = 1900
 hospitalizations[4,1] : Y[4,1] + 0.72 W[4,1] = 0.9                                     hospitalizatio

In [167]:
@constraint(model, vaccinated[i in n_states,j in n_weeks_2], W[i,j+1] == W[i,j] + X[i,j]/ P[i] ) 

2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 1:19
And data, a 50×19 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 vaccinated[1,1] : -1.8522943443896782e-6 X[1,1] - W[1,1] + W[1,2] = 0      …  vaccinated[1,19] : -1.8522943443896782e-6 X[1,19] - W[1,19] + W[1,20] = 0
 vaccinated[2,1] : -7.3507251490359526e-6 X[2,1] - W[2,1] + W[2,2] = 0         vaccinated[2,19] : -7.3507251490359526e-6 X[2,19] - W[2,19] + W[2,20] = 0
 vaccinated[3,1] : -1.2555920932854701e-6 X[3,1] - W[3,1] + W[3,2] = 0         vaccinated[3,19] : -1.2555920932854701e-6 X[3,19] - W[3,19] + W[3,20] = 0
 vaccinated[4,1] : -3.0014887384142533e-6 X[4,1] - W[4,1] + W[4,2] = 0         vaccinated[4,19] : -3.00148873841425

In [168]:
# Initial values of W are  0 
@constraint(model, initial[i in n_states], W[i,1] == 0) 

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:50
And data, a 50-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 initial[1] : W[1,1] = 0
 initial[2] : W[2,1] = 0
 initial[3] : W[3,1] = 0
 initial[4] : W[4,1] = 0
 initial[5] : W[5,1] = 0
 initial[6] : W[6,1] = 0
 initial[7] : W[7,1] = 0
 initial[8] : W[8,1] = 0
 initial[9] : W[9,1] = 0
 initial[10] : W[10,1] = 0
 initial[11] : W[11,1] = 0
 initial[12] : W[12,1] = 0
 initial[13] : W[13,1] = 0
 ⋮
 initial[39] : W[39,1] = 0
 initial[40] : W[40,1] = 0
 initial[41] : W[41,1] = 0
 initial[42] : W[42,1] = 0
 initial[43] : W[43,1] = 0
 initial[44] : W[44,1] = 0
 initial[45] : W[45,1] = 0
 initial[46] : W[46,1] = 0
 initial[47] : W[47,1] = 0
 initial[48]

In [169]:
# W should be between 0 and 1
@constraint(model, bounds[i in n_states,j in n_weeks], W[i,j] <= 1)

2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 1:20
And data, a 50×20 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 bounds[1,1] : W[1,1] ≤ 1    …  bounds[1,20] : W[1,20] ≤ 1
 bounds[2,1] : W[2,1] ≤ 1       bounds[2,20] : W[2,20] ≤ 1
 bounds[3,1] : W[3,1] ≤ 1       bounds[3,20] : W[3,20] ≤ 1
 bounds[4,1] : W[4,1] ≤ 1       bounds[4,20] : W[4,20] ≤ 1
 bounds[5,1] : W[5,1] ≤ 1       bounds[5,20] : W[5,20] ≤ 1
 bounds[6,1] : W[6,1] ≤ 1    …  bounds[6,20] : W[6,20] ≤ 1
 bounds[7,1] : W[7,1] ≤ 1       bounds[7,20] : W[7,20] ≤ 1
 bounds[8,1] : W[8,1] ≤ 1       bounds[8,20] : W[8,20] ≤ 1
 bounds[9,1] : W[9,1] ≤ 1       bounds[9,20] : W[9,20] ≤ 1
 bounds[10,1] : W[10,1] ≤ 1     bounds[10

In [170]:
@constraint(model,vaccine_supply[j in n_weeks], sum(X[i,j] for i in n_states) <= V[j]) 

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:20
And data, a 20-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 vaccine_supply[1] : X[1,1] + X[2,1] + X[3,1] + X[4,1] + X[5,1] + X[6,1] + X[7,1] + X[8,1] + X[9,1] + X[10,1] + X[11,1] + X[12,1] + X[13,1] + X[14,1] + X[15,1] + X[16,1] + X[17,1] + X[18,1] + X[19,1] + X[20,1] + X[21,1] + X[22,1] + X[23,1] + X[24,1] + X[25,1] + X[26,1] + X[27,1] + X[28,1] + X[29,1] + X[30,1] + X[31,1] + X[32,1] + X[33,1] + X[34,1] + X[35,1] + X[36,1] + X[37,1] + X[38,1] + X[39,1] + X[40,1] + X[41,1] + X[42,1] + X[43,1] + X[44,1] + X[45,1] + X[46,1] + X[47,1] + X[48,1] + X[49,1] + X[50,1] ≤ 268301
 vaccine_supply[2] : X[1,2] + X[2,2] + X[3,2] + X[4,2] + X[5,2] + X[6,

In [171]:
@constraint(model,state_vaccination_capacity[i in n_states, j in n_weeks], X[i,j] <= gamma * P[i])  

2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 1:20
And data, a 50×20 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 state_vaccination_capacity[1,1] : X[1,1] ≤ 53987.100000000006    …  state_vaccination_capacity[1,20] : X[1,20] ≤ 53987.100000000006
 state_vaccination_capacity[2,1] : X[2,1] ≤ 13604.1                  state_vaccination_capacity[2,20] : X[2,20] ≤ 13604.1
 state_vaccination_capacity[3,1] : X[3,1] ≤ 79643.70000000001        state_vaccination_capacity[3,20] : X[3,20] ≤ 79643.70000000001
 state_vaccination_capacity[4,1] : X[4,1] ≤ 33316.8                  state_vaccination_capacity[4,20] : X[4,20] ≤ 33316.8
 state_vaccination_capacity[5,1] : X[5,1] ≤ 16174              

In [172]:
optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-1060NG7 CPU @ 1.20GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4020 rows, 3000 columns and 7862 nonzeros
Model fingerprint: 0xddaec6d5
Coefficient statistics:
  Matrix range     [1e-06, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 7e+05]
Presolve removed 3951 rows and 2017 columns
Presolve time: 0.00s
Presolved: 69 rows, 983 columns, 1916 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5128571e+05   5.914745e+06   0.000000e+00      0s
     218    6.5744701e+05   0.000000e+00   0.000000e+00      0s

Solved in 218 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.574470091e+05

User-callback calls 297, time in user-callback 0.00 sec


In [173]:
objective_value(model)


657447.0091388986

In [174]:
W = value.(W)
X = value.(X)
Y = value.(Y)

X_soln = zeros(total_states, total_weeks)
Y_soln = zeros(total_states, total_weeks)
W_soln = zeros(total_states, total_weeks)
# Compute distances between all pairs of points
for i in 1:50
    for j in 1:20
        X_soln[i,j] = X[i,j]
        Y_soln[i,j] = Y[i,j]
        W_soln[i,j] = W[i,j]
    end
end

# save the solutions as a jld file
save("solution.jld", "X_soln", X_soln, "Y_soln", Y_soln, "W_soln", W_soln)